In [1]:
import numpy as np
import pandas as pd
import math
import time
import matplotlib.pyplot as plt

In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [158]:
from sklearn import ensemble

In [3]:
data = pd.read_csv('LoanStats_2016Q3.csv', sep=',', header=1)
intro = pd.read_excel('LCDataDictionary.xlsx')

/var/folders/3v/m7ykfptx4bn621t68jmzkl580000gn/T/ipykernel_2096/787853019.py:1: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('LoanStats_2016Q3.csv', sep=',', header=1)


In [4]:
data = data.drop([99120, 99121], axis = 0)

In [5]:
data = data.dropna(how='all', axis = 1)

In [6]:
data = data.dropna(how='all', axis = 0)

In [8]:
data = data.drop('funded_amnt', axis = 1)

In [9]:
data = data.drop('emp_title', axis = 1)

In [10]:
data['emp_length'] = data['emp_length'].fillna(0)

In [11]:
data['emp_length'] = data['emp_length'].replace(to_replace='[^0-9]+', value='', regex=True)

In [12]:
data['emp_length'] = data['emp_length'].astype('int')

In [13]:
data.columns

Index(['loan_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment',
       'grade', 'sub_grade', 'emp_length', 'home_ownership', 'annual_inc',
       ...
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit'],
      dtype='object', length=106)

In [17]:
data.select_dtypes('object').describe().T

,count,unique,top,freq
term,99120,2,36 months,73898
int_rate,99120,49,11.49%,8316
grade,99120,7,C,32846
sub_grade,99120,35,B5,8322
home_ownership,99120,4,MORTGAGE,46761
verification_status,99120,3,Source Verified,40781
issue_d,99120,3,Aug-2016,36280
loan_status,99120,7,Current,79445
pymnt_plan,99120,2,n,99074
desc,6,5,,2


In [19]:
data = data.drop(['desc', 'verification_status_joint'], axis = 1)

In [21]:
data = data.drop(['zip_code', 'addr_state', 'earliest_cr_line', 'revol_util'], axis = 1)

In [22]:
data = data.drop(['purpose', 'title', 'term', 'issue_d'], axis = 1)

In [26]:
data.drop(['out_prncp','out_prncp_inv','total_pymnt',
         'total_pymnt_inv','total_rec_prncp', 'grade', 'sub_grade'], axis = 1, inplace=True)
data.drop(['total_rec_int','total_rec_late_fee',
         'recoveries','collection_recovery_fee',
         'collection_recovery_fee' ], axis = 1, inplace=True)
data.drop(['last_pymnt_d','last_pymnt_amnt',
         'next_pymnt_d','last_credit_pull_d'], axis = 1, inplace=True)
data.drop(['policy_code'], axis = 1, inplace=True)

In [27]:
data.shape

(99120, 80)

In [33]:
data['int_rate'] = data['int_rate'].str.replace('%', '').astype('float')

In [39]:
data.select_dtypes('float').describe().T.\
assign(missing_pct = data.select_dtypes('float').apply(lambda x: (len(x)-x.count())/len(x)))

,count,mean,std,min,25%,50%,75%,max,missing_pct
loan_amnt,99120.0,14170.570521,8886.138758,1000.00,7200.00,12000.00,20000.00,40000.00,0.0
funded_amnt_inv,99120.0,14166.087823,8883.301328,1000.00,7200.00,12000.00,20000.00,40000.00,0.0
int_rate,99120.0,13.723641,4.873910,5.32,10.49,12.79,15.59,30.99,0.0
installment,99120.0,432.718654,272.678596,30.12,235.24,361.38,569.83,1535.71,0.0
annual_inc,99120.0,78488.850081,72694.186060,0.00,48000.00,65448.00,94000.00,8400000.00,0.0
...,...,...,...,...,...,...,...,...,...
tax_liens,99120.0,0.075393,0.517275,0.00,0.00,0.00,0.00,61.00,0.0
tot_hi_cred_lim,99120.0,172185.283394,175273.669652,2500.00,49130.75,108020.50,248473.25,3953111.00,0.0
total_bal_ex_mort,99120.0,50818.694078,48976.640478,0.00,20913.00,37747.50,64216.25,1548128.00,0.0
total_bc_limit,99120.0,20862.228420,20721.900664,0.00,7700.00,14700.00,27000.00,520500.00,0.0


In [58]:
temp = data.select_dtypes('float').describe().T[data.select_dtypes('float').describe().T['count']<2000].index.tolist()

In [71]:
data.select_dtypes('float').describe().head()

,loan_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,99120.000000,99120.000000,99120.000000,99120.000000,99120.000000,99120.000000,99120.000000,99120.000000,53366.000000,19792.000000,...,99120.000000,99120.000000,99120.000000,98006.000000,99120.000000,99120.000000,99120.000000,99120.000000,99120.000000,99120.000000
mean,14170.570521,14166.087823,13.723641,432.718654,78488.850081,18.348651,0.381901,0.570521,33.229172,67.267886,...,0.101332,2.254752,93.262828,42.681332,0.150262,0.075393,172185.283394,50818.694078,20862.228420,44066.340375
std,8886.138758,8883.301328,4.873910,272.678596,72694.186060,64.057603,0.988996,0.863796,21.820407,24.379343,...,0.567112,1.960084,9.696646,36.296425,0.407706,0.517275,175273.669652,48976.640478,20721.900664,44473.458730
min,1000.000000,1000.000000,5.320000,30.120000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2500.000000,0.000000,0.000000,0.000000
25%,7200.000000,7200.000000,10.490000,235.240000,48000.000000,11.910000,0.000000,0.000000,15.000000,52.000000,...,0.000000,1.000000,90.000000,0.000000,0.000000,0.000000,49130.750000,20913.000000,7700.000000,15750.000000


In [60]:
data = data.drop(temp, axis = 1)

In [78]:
data['loan_status'].unique().shape

(7,)

In [80]:
for item in data['loan_status'].unique().tolist():
    if item in ['Fully Paid', 'Current']:
        data['loan_status'] = data['loan_status'].replace(item, int(1))
    elif 'Late' in item:
        data['loan_status'] = data['loan_status'].replace(item, int(0))
    else:
        data['loan_status'] = data['loan_status'].replace(item, np.nan)

In [81]:
data['loan_status'].value_counts()

loan_status
1.0    92511
0.0     2699
Name: count, dtype: int64

In [126]:
cor = data.select_dtypes(['float', 'int']).corr()

In [127]:
cor.loc[:,:] = np.tril(cor, k=-1)

In [128]:
cor = cor.stack()

In [140]:
hi_cor_list = cor[(cor > 0.9) | (cor < -0.9)].sort_values(ascending=False)

In [148]:
name_list = []
for item in hi_cor_list.index:
    if item[0] not in name_list:
        if item[0] in data.columns:
            name_list.append(item[0])

In [149]:
name_list

['funded_amnt_inv',
 'num_sats',
 'tot_hi_cred_lim',
 'num_rev_tl_bal_gt_0',
 'total_il_high_credit_limit',
 'installment',
 'total_bal_ex_mort']

In [151]:
data = data.drop(name_list, axis = 1)

In [153]:
data.dtypes[data.dtypes == object]

home_ownership         object
verification_status    object
pymnt_plan             object
initial_list_status    object
application_type       object
dtype: object

In [177]:
data = data.fillna(-1)

In [177]:
data = data.fillna(-1)

In [178]:
X = data.drop('loan_status', axis = 1)
Y = data['loan_status']
X = pd.get_dummies(X)

In [179]:
X.shape

(99120, 79)

In [180]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state= 99, shuffle=True)

In [181]:
param_grid = {'learning_rate': [0.1, 0.05, 0.02, 0.01],
              'max_depth': [1,2,3,4],
              'min_samples_split': [50,100,200,400],
              'n_estimators': [100,200,400,800]
              }

In [162]:
grid = GridSearchCV(ensemble.GradientBoostingRegressor(), param_grid, n_jobs=4, refit=True)

In [ ]:
grid_class.fit(x_train, y_train)

In [183]:
grid.best_params_

{'learning_rate': 0.02,
 'max_depth': 2,
 'min_samples_split': 400,
 'n_estimators': 800}

In [184]:
grid.best_index_

159

In [185]:
grid.best_score_

0.03298888683432986

In [187]:
grid.best_estimator_.predict(x_test)

array([0.84229392, 0.96175584, 0.96189357, ..., 0.63663272, 0.91668012,
       0.88254282])

In [188]:
y_test

93153    1.0
27880    1.0
59859    1.0
16739    1.0
71587    1.0
        ... 
37180    1.0
31302    1.0
84776    1.0
51780    1.0
88123    1.0
Name: loan_status, Length: 29736, dtype: float64

In [189]:
grid_class = GridSearchCV(ensemble.GradientBoostingClassifier(), param_grid, n_jobs=4, refit=True)

In [ ]:
grid_class.fit(x_train, y_train)